In [7]:
# main_detector.py

import cv2
import torch
import torch.nn as nn
from torchvision import models, transforms
from tensorflow.keras.models import load_model
from PIL import Image
import numpy as np
import mediapipe as mp
import time

# --- 1. KONFIGURASI DAN INISIALISASI ---

# -- Konfigurasi Model --
EYE_MODEL_PATH = 'model/eye_status_model.h5'
YAWN_MODEL_PATH = 'model/yawn_detector_notebook.pth'
YAWN_CLASS_NAMES = ['no_yawn', 'yawn']
EYE_CLASS_LABELS = ['Mata Tertutup', 'Mata Terbuka']

# -- Konfigurasi Deteksi --
# Threshold untuk peringatan
HEAD_DOWN_THRESHOLD = 18.0  # Derajat untuk menunduk
HEAD_TILT_THRESHOLD = 20.0  # Derajat untuk miring ke samping
YAWN_CONFIDENCE_THRESHOLD = 0.8 # Keyakinan minimum untuk deteksi menguap
BLINK_DURATION_THRESHOLD = 2 # Detik, durasi mata tertutup yang dianggap microsleep
CONSECUTIVE_FRAMES_THRESHOLD = 15 # Jumlah frame berturut-turut untuk memicu peringatan

# -- Inisialisasi Library --
# MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)

# -- Muat Model --
print("Memuat model...")
# Model Status Mata (TensorFlow/Keras)
try:
    eye_model = load_model(EYE_MODEL_PATH)
    print("Model status mata berhasil dimuat.")
except Exception as e:
    print(f"Error memuat model mata: {e}")
    exit()

# Model Deteksi Menguap (PyTorch)
try:
    device = torch.device("cpu")
    yawn_model = models.mobilenet_v2()
    num_ftrs = yawn_model.classifier[1].in_features
    yawn_model.classifier[1] = nn.Linear(num_ftrs, len(YAWN_CLASS_NAMES))
    yawn_model.load_state_dict(torch.load(YAWN_MODEL_PATH, map_location=device))
    yawn_model.to(device)
    yawn_model.eval()
    print("Model deteksi menguap berhasil dimuat.")
except Exception as e:
    print(f"Error memuat model menguap: {e}")
    exit()

# Transformasi gambar untuk model menguap
yawn_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Landmark mata dari MediaPipe
LEFT_EYE_IDXS = [362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385, 384, 398]
RIGHT_EYE_IDXS = [33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161, 246]

# --- 2. FUNGSI HELPER ---

def get_head_pose(landmarks, img_w, img_h):
    """Menghitung sudut pose kepala (pitch, yaw, roll)."""
    face_3d = np.array([
        (0.0, 0.0, 0.0),            # Nose tip
        (0.0, -330.0, -65.0),       # Chin
        (-225.0, 170.0, -135.0),    # Left eye left corner
        (225.0, 170.0, -135.0),     # Right eye right corner
        (-150.0, -150.0, -125.0),   # Left Mouth corner
        (150.0, -150.0, -125.0)     # Right mouth corner
    ], dtype=np.float64)

    face_2d = np.array([
        (landmarks[1].x * img_w, landmarks[1].y * img_h),
        (landmarks[175].x * img_w, landmarks[175].y * img_h),
        (landmarks[33].x * img_w, landmarks[33].y * img_h),
        (landmarks[263].x * img_w, landmarks[263].y * img_h),
        (landmarks[61].x * img_w, landmarks[61].y * img_h),
        (landmarks[291].x * img_w, landmarks[291].y * img_h)
    ], dtype=np.float64)

    focal_length = img_w
    center = (img_w / 2, img_h / 2)
    camera_matrix = np.array(
        [[focal_length, 0, center[0]],
         [0, focal_length, center[1]],
         [0, 0, 1]], dtype="double"
    )
    dist_coeffs = np.zeros((4, 1))

    success, rotation_vector, _ = cv2.solvePnP(
        face_3d, face_2d, camera_matrix, dist_coeffs, flags=cv2.SOLVEPNP_ITERATIVE
    )
    
    rmat, _ = cv2.Rodrigues(rotation_vector)
    # Pitch: menunduk/menengadah, Yaw: menoleh, Roll: memiringkan kepala
    angles, _, _, _, _, _ = cv2.RQDecomp3x3(rmat)
    # Konversi ke rentang yang lebih intuitif
    pitch = angles[0] * -1
    yaw = angles[1]
    roll = angles[2] * -1

    return pitch, yaw, roll

def predict_eye_status(eye_img):
    """Memprediksi status mata (terbuka/tertutup) dari gambar mata."""
    if eye_img.size == 0: return 0.0
    gray_eye = cv2.cvtColor(eye_img, cv2.COLOR_BGR2GRAY)
    resized_eye = cv2.resize(gray_eye, (24, 24))
    normalized_eye = resized_eye / 255.0
    input_eye = np.expand_dims(np.expand_dims(normalized_eye, axis=-1), axis=0)
    prediction = eye_model.predict(input_eye, verbose=0)
    return prediction[0][0]

def predict_yawn(face_img):
    """Memprediksi apakah subjek sedang menguap."""
    if face_img.size == 0: return 'no_yawn', 0.0
    rgb_frame = cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)
    pil_image = Image.fromarray(rgb_frame)
    input_tensor = yawn_transform(pil_image).unsqueeze(0).to(device)
    with torch.no_grad():
        outputs = yawn_model(input_tensor)
        probabilities = torch.nn.functional.softmax(outputs[0], dim=0)
        _, preds = torch.max(outputs, 1)
        predicted_class = YAWN_CLASS_NAMES[preds.item()]
        confidence = probabilities[preds.item()].item()
    return predicted_class, confidence

# --- 3. LOOP UTAMA DETEKSI ---

# Variabel untuk melacak status dari waktu ke waktu
head_down_counter = 0
head_tilt_counter = 0
yawn_counter = 0
microsleep_alert = False

left_eye_close_start = None
right_eye_close_start = None
left_blink_duration = 0
right_blink_duration = 0

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Tidak bisa membuka webcam.")
    exit()

print("\nWebcam aktif. Tekan 'q' untuk keluar.")

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    frame = cv2.flip(frame, 1)
    h, w, _ = frame.shape
    
    # Inisialisasi status frame ini
    is_drowsy = False
    microsleep_alert = False
    drowsy_indicators = []

    # Konversi ke RGB untuk MediaPipe
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb_frame)

    if results.multi_face_landmarks:
        landmarks = results.multi_face_landmarks[0].landmark

        # -- 1. Deteksi Pose Kepala --
        pitch, yaw, roll = get_head_pose(landmarks, w, h)
        
        if pitch > HEAD_DOWN_THRESHOLD:
            head_down_counter += 1
            drowsy_indicators.append("Menunduk")
        else:
            head_down_counter = 0

        if abs(roll) > HEAD_TILT_THRESHOLD:
            head_tilt_counter += 1
            drowsy_indicators.append("Kepala Miring")
        else:
            head_tilt_counter = 0

        # -- 2. Deteksi Menguap (Yawn) --
        face_points = np.array([[lm.x * w, lm.y * h] for lm in landmarks]).astype(int)
        x_min, y_min = np.min(face_points, axis=0)
        x_max, y_max = np.max(face_points, axis=0)
        padding = 20
        face_crop = frame[max(0, y_min - padding):min(h, y_max + padding), 
                          max(0, x_min - padding):min(w, x_max + padding)]

        yawn_status, yawn_conf = predict_yawn(face_crop)
        if yawn_status == 'yawn' and yawn_conf > YAWN_CONFIDENCE_THRESHOLD:
            yawn_counter += 1
            drowsy_indicators.append("Menguap")
        else:
            yawn_counter = 0
        
        # -- 3. Deteksi Status Mata & Microsleep --
        # Proses Mata Kiri
        left_eye_points = np.array([[landmarks[i].x * w, landmarks[i].y * h] for i in LEFT_EYE_IDXS]).astype(int)
        lx_min, ly_min = np.min(left_eye_points, axis=0)
        lx_max, ly_max = np.max(left_eye_points, axis=0)
        left_eye_crop = frame[ly_min-5:ly_max+5, lx_min-5:lx_max+5]
        prob_left = predict_eye_status(left_eye_crop)
        status_left = EYE_CLASS_LABELS[1] if prob_left > 0.5 else EYE_CLASS_LABELS[0]

        # Proses Mata Kanan
        right_eye_points = np.array([[landmarks[i].x * w, landmarks[i].y * h] for i in RIGHT_EYE_IDXS]).astype(int)
        rx_min, ry_min = np.min(right_eye_points, axis=0)
        rx_max, ry_max = np.max(right_eye_points, axis=0)
        right_eye_crop = frame[ry_min-5:ry_max+5, rx_min-5:rx_max+5]
        prob_right = predict_eye_status(right_eye_crop)
        status_right = EYE_CLASS_LABELS[1] if prob_right > 0.5 else EYE_CLASS_LABELS[0]

        # Logika Microsleep
        # Cek mata kiri
        if status_left == 'Mata Tertutup':
            if left_eye_close_start is None:
                left_eye_close_start = time.time()
            else:
                left_blink_duration = time.time() - left_eye_close_start
        else: # Mata terbuka
            if left_eye_close_start is not None:
                left_blink_duration = time.time() - left_eye_close_start
            left_eye_close_start = None

        # Cek mata kanan
        if status_right == 'Mata Tertutup':
            if right_eye_close_start is None:
                right_eye_close_start = time.time()
            else:
                right_blink_duration = time.time() - right_eye_close_start
        else: # Mata terbuka
            if right_eye_close_start is not None:
                right_blink_duration = time.time() - right_eye_close_start
            right_eye_close_start = None
            
        # Peringatan Microsleep
        if left_blink_duration > BLINK_DURATION_THRESHOLD or right_blink_duration > BLINK_DURATION_THRESHOLD:
             microsleep_alert = True
             drowsy_indicators.append("Kedipan Lambat")

        if (status_left == 'Mata Tertutup' and status_right == 'Mata Terbuka') or \
           (status_right == 'Mata Tertutup' and status_left == 'Mata Terbuka'):
            drowsy_indicators.append("Mata Tertutup Sebelah")
            microsleep_alert = True


        # -- 4. Agregasi Status & Tampilan --
        # Tentukan status kantuk akhir
        if head_down_counter > CONSECUTIVE_FRAMES_THRESHOLD or \
           head_tilt_counter > CONSECUTIVE_FRAMES_THRESHOLD or \
           yawn_counter > (CONSECUTIVE_FRAMES_THRESHOLD / 2) or \
           microsleep_alert:
            is_drowsy = True
        
        # Gambar Bounding Box Wajah
        bbox_color = (0, 0, 255) if is_drowsy else (0, 255, 0)
        cv2.rectangle(frame, (x_min - padding, y_min - padding), (x_max + padding, y_max + padding), bbox_color, 2)

        # Gambar status utama
        status_text = "MENGANTUK!" if is_drowsy else "NORMAL"
        cv2.putText(frame, status_text, (x_min - padding, y_min - padding - 15), cv2.FONT_HERSHEY_SIMPLEX, 0.9, bbox_color, 2)
        
        # Gambar status mata
        color_left = (0, 255, 0) if status_left == 'Mata Terbuka' else (0, 0, 255)
        color_right = (0, 255, 0) if status_right == 'Mata Terbuka' else (0, 0, 255)
        cv2.rectangle(frame, (lx_min-5, ly_min-5), (lx_max+5, ly_max+5), color_left, 1)
        cv2.rectangle(frame, (rx_min-5, ry_min-5), (rx_max+5, ry_max+5), color_right, 1)

        # Tampilkan informasi detail di layar dengan warna BIRU
        info_y = 30
        text_color_blue = (255, 0, 0) # BGR format untuk warna biru
        cv2.putText(frame, f"Pitch (Tunduk): {pitch:.1f}", (10, info_y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, text_color_blue, 2)
        cv2.putText(frame, f"Roll (Miring): {roll:.1f}", (10, info_y + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.6, text_color_blue, 2)
        cv2.putText(frame, f"Status Menguap: {yawn_status} ({yawn_conf:.2f})", (10, info_y + 50), cv2.FONT_HERSHEY_SIMPLEX, 0.6, text_color_blue, 2)
        cv2.putText(frame, f"Durasi Kedip: {max(left_blink_duration, right_blink_duration):.2f}s", (10, info_y + 75), cv2.FONT_HERSHEY_SIMPLEX, 0.6, text_color_blue, 2)

        if drowsy_indicators:
            indicators_text = ", ".join(list(set(drowsy_indicators)))
            cv2.putText(frame, f"Indikator: {indicators_text}", (10, h - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)

        if microsleep_alert:
            cv2.putText(frame, "MICROSLEEP DETECTED!", (w // 2 - 150, h // 2 - 50), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 255), 3)

        if is_drowsy:
            overlay = frame.copy()
            cv2.rectangle(overlay, (0, 0), (w, h), (0, 0, 255), -1)
            frame = cv2.addWeighted(frame, 0.7, overlay, 0.3, 0)
            cv2.putText(frame, "BAHAYA: KANTUK TERDETEKSI!", (w // 2 - 300, h // 2), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255), 3)

        # Reset durasi kedip jika mata sudah terbuka kembali
        if status_left == 'Mata Terbuka': left_blink_duration = 0
        if status_right == 'Mata Terbuka': right_blink_duration = 0

    cv2.imshow('Driver Drowsiness Detection', frame)

    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
face_mesh.close()

Memuat model...
Model status mata berhasil dimuat.
Model deteksi menguap berhasil dimuat.

Webcam aktif. Tekan 'q' untuk keluar.
